Pseudo skeleton code for ACO algorithm applied to TSP (Mohsin,Thangavelu. (2021))

Introduction to the solution 
%Ant system for solving the traveling salesman problem:
<Input of matrix D of distances>
<Initialization of the parameters of the algorithm: α, β, Q, p, and τ0>
m=n %Number of ants is equal to the number of towns
For k=1:m
    <Allocate ant k in randomly chosen town>
End
<Select a conditionally shortest route T+ and calculate its length L+>;
%Main loop
For t=1:tma
 %For each ant:
    <Build a route Tk(t) according to rule (1)>
    <Calculate Lk(t), the length of the route Tk(t)>
    End
    If “Is the best solution found?” <Update T+ and L+> End
    For i=1:n
        For j=1:n %For each edge
            <Update pheromone trails according to rule (2)>
        End
    End
End

Sections
Read tsp file 
Create LIST with the cities and coordinates
create ARRAY with distances between cities (the number of distances is a combination nCr 323 distancias)

Questions:
Whats the answer format? And array with the rute example city1, city4, city5....cityn

In [193]:
##import libraries
import math
import pandas as pd
import numpy as np

following code shows script to read the content of the tsp file and store it in a list and each index is a tuple that contains city-n, x,y
[( ),( ),( ),( ),....................,( )]
#from now on we are going to address the cities by their index in the list/Array/matrix instead of their city collum value to prevent confusions 
index[n-1]   city_n    coordinatesX     coordinatesY
0               1       11003.611100    42102.500000           

In [201]:
# Open input file
cities = [] #create array to store cities
file = open('dj38.tsp', 'r') #read file
lines = file.readlines() #read lines
for line in (lines[10:]): #loop over lines
    # Assumes the file format: <CityIndex> <X> <Y>
    city_index, x, y = map(float, line.strip().split()) #separate elements by blank space
    cities.append((city_index, x, y)) #creates a list with al the elements city, x, y

Following code shows the calculation of distances between the cities, visibility, and matrixes fill up 
distance_func: Function that calculates the distance between two nodes/points/Cities given two cities (i,j)
visibility_func: function that calculates the visibility between two nodes/points/cities given two cities(i,j)

distances: (D_ij) matrix that stores all distances between nodes
visibility: (η_ij) matrix that stores all visibilities between nodes
pheromones_t:  (𝜏_ij(t)) matrix that stores pheromone strenght (number of virtual pheromone on the edge i,j) between nodes at t time (iteration) changes with each iteration

In [312]:
def distance_func(city_i,city_j): #city i is current city while city j is the next city
    x1=cities[city_i][1] #identify x1 coordinate
    x2=cities[city_j][1] #identify x2 coordinate
    y1=cities[city_i][2] #identify y1 coordinate
    y2=cities[city_j][2] #identify y2 coordinate
    d=math.sqrt((x2-x1)**2+(y2-y1)**2) ##apply formula for the distance between two points
    return d

def visibility_func(city_i,city_j): #city i is current city while city j is the next city
    if distances[city_i][city_j]==0:
        nij=0
    else:
        nij=1/distance_func(city_i,city_j) #distances[city_i][city_j] ##apply formula for the distance between two points
    return nij

#matrixes declaration
distances = np.array([np.array([distance_func(i,j) for i in range(len(cities))]) for j in range(len(cities))])
visibilities = np.array([np.array([visibility_func(i,j) for i in range(len(cities))])for j in range(len(cities))])
pheromones_t = np.ones((len(cities),len(cities))) #𝜏_ij
print(distances[10][37])


1216.421866016664


following codes shows initial solution for ONE ant

In [316]:
alpha = .01                                                                   #pheromone value(how likely is to take into account pheromone value)
beta = .01                                                                    #heuristic value(how likely is to take randomness into account?)
p = .05                                                                       #pheromone evaporation coefficient
n=len(cities)                                                                 
k=n                                                                           #number of ants given the number of cities

for ant_k in range (len(cities)):
      tabu_list_k = np.array([x for x in range (len(cities))])                      #list of unvisited cities for ant k

      #Ant_k starts in city i (same value as ant) and then goes on to j and so on until it completes a route and then goes Ant_1....and so on until Ant_n
      i=ant_k                                                                           #starts at city 0
      x=0
      tabu_list_k = np.delete(tabu_list_k,np.where(tabu_list_k==i))                 #deletes from the tabu list on this ant k the city they are visiting
      while i not in tabu_list_k:                                                   #while ant at city (i) not in the list of not visited cities decide which city visit next
            best_option=(0,0)                                                       #tuple(probability_ij, j) place holder for the city who has the better probability during the iterations
            for j in tabu_list_k:                                                   #chooses next city (trans: starts a decision loop to choose next city (j))
                  probability_il = 0                                                #sets probability for not visited cities to 0
                  for l in tabu_list_k:                                             #loops over all the cities not visited 
                        probability_il = probability_il + (pheromones_t[i][l]*alpha*(visibilities[i][l])*beta)
                  probability_ij = (pheromones_t[i][j]*alpha*(1/distance_func(i,j))*beta) /probability_il #probability of visiting next city
                  if probability_ij >= best_option[0]:                              #compares the probability for each city j at  city i
                        best_option=(probability_ij,j)                              #if meets condition new best option is set

            print(i,best_option)                                                   #from, distance, to  
            i=best_option[1]                                                        #new city that is going to be visited
            tabu_list_k = np.delete(tabu_list_k,np.where(tabu_list_k==i))           #deletes from the tabu list on this ant k the city they are visiting
            if len(tabu_list_k)==0:
                  break


0 (0.0888265338888701, 1)
1 (0.06411502647427113, 3)
3 (0.09088240319903337, 2)
2 (0.20150891566531973, 4)
4 (0.11922868168488987, 6)
6 (0.15735029048553592, 5)
5 (0.09139685153800867, 7)
7 (0.44561313783850476, 8)
8 (0.0880069947384692, 12)
12 (0.4311853720415107, 14)
14 (0.07552158699068211, 19)
19 (0.16310277832715267, 22)
22 (0.23533572790577847, 24)
24 (0.4420836239931347, 25)
25 (0.12288831321399273, 21)
21 (0.25232770951264594, 23)
23 (0.1030753124762918, 27)
27 (0.09669880705343899, 26)
26 (0.0890947591624176, 30)
30 (0.2552742639554792, 35)
35 (0.2227982532938889, 33)
33 (0.5821747420103298, 32)
32 (0.16705503989112558, 36)
36 (0.2203564485489113, 34)
34 (0.1739204311819537, 31)
31 (0.3180936185141158, 29)
29 (0.22160813391319809, 28)
28 (0.16945963924183702, 20)
20 (0.27039408201658205, 13)
13 (0.6419493247491254, 9)
9 (0.15276532956030425, 15)
15 (0.3844448953607586, 16)
16 (0.5241748569540818, 17)
17 (0.9235715658164775, 18)
18 (0.5524647954442756, 11)
11 (0.943982141148027

In [321]:
x=37*38
print(x)

1406


BIO
MOHAMED Mohsin, AJANTH Thangavelu
On Embarrassingly Parallel MaxMin Ant Colony Optimization for
Traveling Salesperson Problem. 2021